- competition/dataset : [https://www.kaggle.com/c/statoil-iceberg-classifier-challenge/overview](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge/overview)
- date : 2021/02/10
- original : [https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712](https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712)

## Transfer Learning with VGG-16 CNN+AUG LB 0.1712

**✏ 필사 1회** 

##### TL;DR
**GPU에서 실행**해야 하며, CPU에서는 호환성의 문제가 있습니다:
1. 딥러닝에서 하이퍼파라미터는 매우 많으며, 튜닝하기 위해서는 몇 주~몇 달이 걸립니다. 일반적으로 연구원들은 튜닝을 수행한 뒤 다른 것보다 우수한 구조의 셋을 발견하면 논문을 발표합니다.
2. 모델에 대해 사전훈련이 되었기 때문에 빠르게 실행되나 여전히 GPU가 필요합니다. 몇 가지 라이브러리의 이슈 때문에 CPU에서는 작동하지 않습니다.
3. 우리의 목적을 위해 연구원들이 우리에게 사용할 수 있도록 한 구조물들을 사용할 수 있습니다.
4. 이미 feature를 추출하는 방법을 알고 있는 미리 훈련된 네트워크와 레이어를 사용하면 하이퍼파라미터를 튜닝하지 않아도 됩니다. 이미 데이터 셋 일부에 대해 훈련되었기 때문에 이미 훈련된 가중치는 좋은 가중치 초기화를 제공하여 우리의 Convnet은 매우 빠르게 수렴되며, 그렇지 않으면 이 심층 구조에 대해 며칠이 걸릴 수 있습니다.
5. 이 커널에서는 사전학습된 VGG-16 네트워크를 사용할 것이며, 이것은 작은 크기의 이미지에 매우 좋습니다.  

**VGG 구조가 작은 크기의 이미지(CIFAR-10)에서 잘 작동한다는 것은 이미 증명되었습니다.** 우리가 가진 데이터 셋에도 잘 작용할 것이라고 기대합니다:
1. 이 코드는 데이터 확대 단계도 포함하고 있기 때문에 상당히 성능이 향상될 것입니다. 
2. GPU는 반드시 필요합니다.

In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [11]:
train = pd.read_json('data/train.json')
test = pd.read_json('data/test.json')

target_train = train['is_iceberg']

keras는 사전훈련된 VGG의 구현을 제공하며, 라이브러리에 있기 때문에 우리가 직접 네트워크를 구축할 필요가 없습니다. 여기서 우리는 VGG의 마지막 레이어는 지우고 이진 분류를 위해 sigmoid 레이어를 넣습니다.  

다음 코드는 kaggle 노트북에서는 모델의 가중치에 대해 다운로드되어 있지 않기 때문에 작동하지 않습니다.

In [3]:
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
x_angle = train['inc_angle']

test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
x_test_angle = test['inc_angle']

In [4]:
x_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
x_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])
x_band_3 = (x_band_1 + x_band_2) / 2
x_train = np.concatenate([
    x_band_1[:, :, :, np.newaxis],
    x_band_2[:, :, :, np.newaxis],
    x_band_3[:, :, :, np.newaxis]
], axis=-1)

In [5]:
x_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
x_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])
x_band_3 = (x_band_1 + x_band_2) / 2
x_test = np.concatenate([
    x_band_1[:, :, :, np.newaxis],
    x_band_2[:, :, :, np.newaxis],
    x_band_3[:, :, :, np.newaxis]
], axis=-1)

In [6]:
from matplotlib import pyplot
from keras.optimizers import RMSprop, Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras import initializers
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, LSTM, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [7]:
# data augmentation(데이터 증가) for multi-input
batch_size = 64
# 이미지 변환 정의
gen = ImageDataGenerator(
    horizontal_flip=True, vertical_flip=True,
    width_shift_range=0., height_shift_range=0., channel_shift_range=0.,
    zoom_range=0.2, rotation_range=10
)

In [8]:
# 두 generator merge하는 함수 정의
# y와 angle array에 대해 동일한 생성자와 동일한 random seed를 사용
def gen_flow_for_two_inputs(x1, x2, y):
    genx1 = gen.flow(x1, y, batch_size=batch_size, seed=55)
    genx2 = gen.flow(x1, x2, batch_size=batch_size, seed=55)
    while True:
        x1i = genx1.next()
        x2i = genx2.next()
        # 두 배열이 동일한지 평가
        # np.testing.assert_array_equal(x1i[0], x2i[0]) # 오래걸림
        yield [x1i[0], x2i[1]], x1i[1]

In [9]:
# generator 생성 함수 정의
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [10]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name='angle')
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(
        weights='imagenet', include_top=False, input_shape=x_train.shape[1:], classes=1
    )
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model([base_model.input, input_2], predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [11]:
# data augmentation에 대한 K-fold 교차검증 함수 생성
def myAngleCV(x_train, x_angle, x_test):
    K = 3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(x_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0 * target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print(f'\n===================FOLD={j}')
        x_train_cv = x_train[train_idx]
        y_train_cv = target_train[train_idx]
        x_holdout = x_train[test_idx]
        y_holdout = target_train[test_idx]
        
        # Angle
        x_angle_cv = x_angle[train_idx]
        x_angle_hold = x_angle[test_idx]
        
        # file paht 지정 및 get callbacks 실행
        file_path = 'data/%s_aug_model_weights.hdf5'%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(x_train_cv, x_angle_cv, y_train_cv)
        galaxyModel = getVggAngleModel()
        galaxyModel.fit_generator(
            gen_flow,
            steps_per_epoch=24, epochs=100,
            shuffle=True, verbose=1,
            validation_data=([x_holdout, x_angle_hold], y_holdout),
            callbacks=callbacks
        )
        
        # 최적 모델
        galaxyModel.load_weights(filepath=file_path)
        
        # 트레이닝 점수
        score = galaxyModel.evaluate([x_train_cv, x_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        
        # 테스트 점수
        score = galaxyModel.evaluate([x_holdout, x_angle_hold], y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        
        # 검증 점수
        pred_valid = galaxyModel.predict([x_holdout, x_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])
        
        # 테스트 점수
        temp_test = galaxyModel.predict([x_test, x_test_angle])
        y_test_pred_log += temp_test.reshape(temp_test.shape[0])
        
        # 트레인 점수
        temp_train = galaxyModel.predict([x_train, x_angle])
        y_train_pred_log += temp_train.reshape(temp_train.shape[0])
    
    y_test_pred_log = y_test_pred_log / K
    y_train_pred_log = y_train_pred_log / K
    print('\n Train Log Loss Validation =', log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation =', log_loss(target_train, y_valid_pred_log))
    return y_train_pred_log

In [ ]:
preds = myAngleCV(x_train, x_angle, x_test)

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test['id']
subsmission['is_iceberg'] = preds
submission.to_csv('data/submission_2_sub.csv', index=False)